# 1. 잠재 디리클레 할당(Latent Dirichlet Allcation, LDA) 개요
LDA는 주어진 데이터의 문서 생성 과정을 역추적    
'문서는 토픽 혼합으로 구성되며, 토픽은 확률분포에 기반해 단어를 생성한다'는 가정   
토픽 제목을 정하진 않으나 결과로부터 토픽 판단 가능   

문서1: 저는 사과랑 바나나를 먹어요   
문서2: 우리는 귀여운 강아지가 좋아요   
문서3: 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요   

<문서 토픽 분포>   
문서1: 토픽 A 100%   
문서2: 토픽 B 100%   
문서3: 토픽 B 60%, 토픽 A 40%   

<토픽 단어 분포>   
토픽A: 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜직하고 0%, 좋아요 0%   
토픽B: 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%   

# 2. LDA의 가정
BoW 행렬 DTM 또는 TF-IDF 행렬을 입력함, 단어 순서는 신경쓰지 않음

> **1) 문서에 사용할 단어 개수 N을 정함**   

> **2) 문서에 사용할 토픽 혼합을 확률분포에 기반하여 결정**   
> 예) 토픽이 2개면 강아지 토픽을 60%, 과일 토픽을 40%로 선택

> **3) 문서에 사용할 각 단어를 정함**   
> 3)을 반복하며 문서 완성

>> **3-1) 토픽 분포에서 토픽 T를 확률적으로 고름**   

>> **3-2) 선택한 토픽 T에서 단어의 출현 확률분포에 기반해 문서에 사용할 단어를 고름**   
>> 예) 강아지 토픽을 선택한다면, 33% 확률로 강아지란 단어 선택

# 3. LDA의 수행하기
> **1) 사용자는 알고리즘에게 토픽 개수 k를 알려줌**   
> 입력된 k개 토픽이 M개의 전체 문서에 걸쳐 분포되었다고 가정

> **2) 모든 단어를 k개 중 하나의 토픽에 할당**   
> 각 문서는 토픽을, 토픽은 단어 분포를 갖게 됨   
> 랜덤으로 할당했기에 결과는 전부 틀림   
> 만약 한 단어가 같은 문서에 2개 이상이라면, 각 단어는 서로 다른 토픽일 수 있음

> **3) 모든 문서의 모든 단어에 대해 아래 사항을 iteration**   
>> **3-1) 각 단어 w 자신은 잘못된 토픽에, 다른 단어들은 올바른 토픽에 할당되었다고 가정**   
>> 단어 w는 아래 기준으로 토픽이 재할당됨   
>> 1. p(topic t|document d): 문서 d의 단어에서 토픽 t에 해당하는 단어 비율   
>> 2. p(word w|topic t): 토픽 t에서 단어 w의 분포   

>> doc1 word|apple|banana|apple|dog|dog
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|B|B|???|A|A

>> doc2 word|cute|book|king|apple|apple
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|B|B|B|B|B

>> 여기서 doc1의 세번째 단어 apple의 토픽을 결정하는 상황   

>> doc1 word|**apple**|**banana**|apple|**dog**|**dog**
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|**B**|**B**|???|**A**|**A**

>> 첫째 기준: doc1 단어들이 어떤 토픽인지 확인   
>> doc1 단어들은 토픽 A와 토픽 B에 50 대 50의 비율로 할당됨   
>> 이에 따라 단어 apple은 어떤 토픽에도 속할 가능성이 있음   

>> doc1 word|**apple**|banana|**apple**|dog|dog
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|**B**|B|**???**|A|A

>> doc2 word|cute|book|king|**apple**|**apple**
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|B|B|B|**B**|**B**

>> 둘째 기준: 전체 문서에서 단어 apple이 어떤 토픽인지 확인   
>> 이에 따라 단어 apple은 토픽 B일 가능성이 높음

# 4. 잠재 의미 분석과 잠재 디리클레 할당의 차이
LSA: DTM의 축소된 차원에서 근접 단어들을 토픽으로 묶음   
LDA: 단어가 특정 토픽에 존재할 확률, 문서에 특정 토픽이 존재할 확률을 결합확률로 추정해 토픽을 추출

# 5. 실습을 통한 이해
> **1) 정수 인코딩과 단어 집합 만들기**   

In [1]:
# 뉴스그룹 데이터 로드
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

# 텍스트 전처리
news_df = pd.DataFrame({'document':documents})

news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

<ipython-input-1-aedc00f00276>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [2]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [3]:
# 정수 인코딩 및 단어 빈도수 기록
from gensim import corpora 
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [4]:
print(dictionary[55])

accept


In [5]:
len(dictionary)

64281

> **2) LDA 모델 훈련시키기**   
> 단어 앞 수치는 토픽에 대한 단어 기여도

In [6]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.059*"space" + 0.022*"nasa" + 0.012*"launch" + 0.011*"earth"')
(1, '0.016*"rules" + 0.010*"water" + 0.009*"picture" + 0.009*"uuencode"')
(2, '0.024*"window" + 0.014*"display" + 0.012*"scsi" + 0.012*"color"')
(3, '0.037*"char" + 0.008*"pointer" + 0.008*"column" + 0.008*"obfuscate"')
(4, '0.014*"game" + 0.013*"team" + 0.012*"year" + 0.009*"games"')
(5, '0.012*"world" + 0.008*"history" + 0.008*"american" + 0.005*"anti"')
(6, '0.014*"gordon" + 0.013*"pitt" + 0.012*"banks" + 0.012*"surrender"')
(7, '0.008*"health" + 0.006*"problem" + 0.006*"pain" + 0.006*"medical"')
(8, '0.012*"people" + 0.011*"would" + 0.008*"think" + 0.006*"believe"')
(9, '0.014*"drive" + 0.010*"card" + 0.010*"system" + 0.009*"disk"')
(10, '0.029*"period" + 0.018*"play" + 0.016*"power" + 0.011*"scorer"')
(11, '0.022*"file" + 0.014*"program" + 0.011*"files" + 0.010*"available"')
(12, '0.019*"ground" + 0.017*"wire" + 0.010*"wiring" + 0.008*"panel"')
(13, '0.009*"government" + 0.008*"would" + 0.008*"people" + 0.006*"pr

> **3) LDA 시각화하기**   
> 모델 출력에서는 토픽 번호가 0으로 할당되나 시각화에서는 1로 시작

In [7]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

> **4) 문서 별 토픽 분포 보기**

In [8]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i == 5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(5, 0.5158921), (8, 0.31494), (18, 0.15545006)]
1 번째 문서의 topic 비율은 [(3, 0.10654157), (4, 0.03951474), (6, 0.028158862), (8, 0.80472296)]
2 번째 문서의 topic 비율은 [(1, 0.11842401), (4, 0.036669537), (13, 0.11929866), (18, 0.36084467), (19, 0.35263765)]
3 번째 문서의 topic 비율은 [(1, 0.016641706), (5, 0.08563962), (7, 0.027726231), (9, 0.10437531), (13, 0.6110526), (19, 0.14357744)]
4 번째 문서의 topic 비율은 [(1, 0.031897567), (4, 0.32736742), (10, 0.33276403), (19, 0.27832624)]


C:\Users\USER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# 데이터프레임 형식 출력
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        
        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), 
                                                            topic_list]), ignore_index=True)
            else:
                break
                
    return(topic_table)

C:\Users\USER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\USER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,5.0,0.5159,"[(5, 0.5158872), (8, 0.31489232), (18, 0.15550..."
1,1,8.0,0.8048,"[(3, 0.106541574), (4, 0.039485656), (6, 0.028..."
2,2,18.0,0.3608,"[(1, 0.11842467), (4, 0.036659155), (13, 0.119..."
3,3,13.0,0.6110,"[(1, 0.016642632), (5, 0.085641645), (7, 0.027..."
4,4,10.0,0.3328,"[(1, 0.03190449), (4, 0.32724723), (10, 0.3327..."
5,5,19.0,0.4500,"[(5, 0.11579089), (8, 0.32595822), (17, 0.0734..."
6,6,9.0,0.5984,"[(7, 0.26781407), (9, 0.59840673), (11, 0.0641..."
7,7,8.0,0.3258,"[(5, 0.033853903), (6, 0.017137613), (7, 0.131..."
8,8,19.0,0.3480,"[(1, 0.20149784), (8, 0.124162145), (13, 0.273..."
9,9,19.0,0.4965,"[(5, 0.019263621), (8, 0.021777987), (9, 0.226..."
